# Euler Maruyama Black Scholes 

In [14]:
import numpy as np
import sdeint


np.random.seed(123)
N = 10000
sigma = 0.02           #long-term average volatility
r     = 0.02           #risk-free interest rate
tau   = 1/4            #time to maturity
S0    = 100            #initial share price
K     = 70             #strike price
y0 = S0
mu = np.log(S0)+ ((r-sigma**2)/2)
tspan = np.linspace(0.0, 1.0/4, N+1)

def f(x, t):
    return mu 

def g(x, t):
    return sigma

S = sdeint.itoEuler(f, g, y0, tspan)
S
V  =  np.exp(-r*tau)*np.maximum(S-K,0)  
AV =  np.mean(V) 
AV

30.429673684059956